In [184]:
global_namespace = Dict()

function metajulia_repl()
    print(global_namespace)
    print(">>", " ")
    ans = Meta.parse(readline())
    my_eval(ans, global_namespace)
end

function my_eval(expr, namespace)
    if !(expr isa Expr)
        if expr isa Symbol
            return namespace[expr]
        else
            return expr
        end
    else
        if expr.head == :block
            for arg in expr.args[begin:end-1]
                if !(arg isa LineNumberNode)
                    my_eval(arg, namespace)
                end
            end
            return !(expr.args[end] isa LineNumberNode) ? my_eval(expr.args[end], namespace) : nothing
        elseif expr.head == :&&
            return reduce(&, [my_eval(expr.args[x], namespace) for x in 1:length(expr.args)])
        elseif expr.head == :||
            return reduce(|, [my_eval(expr.args[x], namespace) for x in 1:length(expr.args)])
        elseif expr.head == :if
            return my_eval(expr.args[1], namespace) ? my_eval(expr.args[2], namespace) : my_eval(expr.args[3], namespace)
        elseif expr.head == :elseif
            return my_eval(expr.args[1].args[2]) ? my_eval(expr.args[2], namespace) : my_eval(expr.args[3], namespace)
        elseif expr.head == :(=)
            if (expr.args[1] isa Expr)
                println("arrived at function definition!")
                # println(expr.args[1].args[1])
                # println(expr.args[1])
                # println(expr.args[2].args[end])
                arguments = ""
                for arg in expr.args[1].args[2:end]
                    arguments = arguments*string(arg)*","
                end
                arguments = arguments[1:end-1]
                func_expression = string("((", arguments, ") -> ",  expr.args[2].args[end], ")")
                namespace[expr.args[1].args[1]] = func_expression
                println("This is the expresson saved in dict: $func_expression")
            else
                namespace[expr.args[1]] = my_eval(expr.args[2], namespace)
            end
        elseif expr.head == :let
            # println("let")
            # println(expr.args[begin])
            # println(expr.args[end])
            let
                scope_dict = copy(namespace)
                for arg in expr.args[begin:end-1]
                    println("top of let for loop")
                    if !(arg isa LineNumberNode)
                        println("Inside if let")
                        println("typeof(arg)")
                        println(typeof(arg))
                        println(arg.head)
                        if length(arg.args) > 0
                            my_eval(arg, scope_dict)
                        end
                    end
                end
                println("After let for loop")
                return !(expr.args[end] isa LineNumberNode) ? my_eval(expr.args[end], scope_dict) : nothing
            end
        elseif expr.head == :call
            if expr.args[1] == :+
                return reduce(+, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :*
                return reduce(*, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :/
                return reduce(/, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :<
                return reduce(<, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :>
                return reduce(>, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] isa Symbol
                println("You have reached a function call!")
                func_expr = namespace[expr.args[1]]
                func_args = expr.args[2:end]
                resolved_func_expr = resolve_function_expr(func_expr, func_args)
                print("resolved_func_expr\n")
                print(resolved_func_expr)
                resolved_func_expr_parsed = Meta.parse(resolved_func_expr)
                return my_eval(resolved_func_expr_parsed, namespace)
            end
        end
    end
end

function resolve_function_expr(expression, argument_values)
    endOfArgList = findfirst(isequal(')'), expression)
    println("expression[3:endOfArgList-1]")
    println(expression[3:endOfArgList-1])
    args = split(expression[3:endOfArgList-1], ',')
    println("These are the arguments:")
    println(args)
    for arg_idx in 1:length(args)
        println("top of for loop")
        expression = replace(expression, args[arg_idx] => argument_values[arg_idx])
    end

    start = findfirst(">", expression)
    expression = "("*expression[start[1]+1: end]
    
    println("end of function")
    return expression
end



resolve_function_expr (generic function with 1 method)

In [187]:
metajulia_repl()

Dict{Any, Any}(:triple => "((a) -> a + a + a)", :sum => "((fun,a,b) -> if a > b\n    0\nelse\n    fun(a) + sum(fun, a + 1, b)\nend)")>> You have reached a function call!
expression[3:endOfArgList-1]
fun,a,b
These are the arguments:
SubString{String}["fun", "a", "b"]
top of for loop
top of for loop
top of for loop
end of function
resolved_func_expr
( if 1 > 10
    0
else
    triple(1) + sum(triple, 1 + 1, 10)
end)You have reached a function call!
expression[3:endOfArgList-1]
a
These are the arguments:
SubString{String}["a"]
top of for loop
end of function
resolved_func_expr
( 1 + 1 + 1)You have reached a function call!
expression[3:endOfArgList-1]
fun,a,b
These are the arguments:
SubString{String}["fun", "a", "b"]
top of for loop
top of for loop
top of for loop
end of function
resolved_func_expr
( if 1 + 1 > 10
    0
else
    triple(1 + 1) + sum(triple, 1 + 1 + 1, 10)
end)You have reached a function call!
expression[3:endOfArgList-1]
a
These are the arguments:
SubString{String}["a"]
top

165

In [61]:
expr = Meta.parse("y(x, f, z) = x+f+z+2")
# string("(", expr.args[1].args[2], " -> ",  expr.args[2].args[end], ")")

# string(string.(expr.args[1].args)[1], string.(expr.args[1].args)[2])
string(string.(expr.args[1].args)[1], string.(expr.args[1].args)[2])

arguments = ""
for arg in expr.args[1].args[2:end]
    arguments = arguments*string(arg)*","
end

arguments = arguments[1:end-1]

func_expression = string("((", arguments, ") -> ",  expr.args[2].args[end], ")")

resolve_function_expr(func_expression, [1, 2, 3])



expression[3:endOfArgList-1]
x,f,z
These are the arguments:
SubString{String}["x", "f", "z"]


:(1 + 2 + 3 + 2)

In [183]:
Meta.parse(global_namespace[:func])

:(a->begin
          #= none:1 =#
          if a > 1
              #= none:2 =#
              2
          else
              #= none:4 =#
              0
          end
      end)

## 